  
[**Dr. Kevin M. Moerman**](mailto:kevin.moerman@nuigalway.ie), _Lecturer in Biomedical Engineering_   

National University of Ireland Galway.   

---

This content is derived from a Jupyter notebook. Jupyter notebooks are a great way to combine teaching of theory and numerical implementation side-by-side and in an interactive manor.

* To run the Jupyter notebooks and to interact with the code uses should set-up Jupyter. Note that doing so is not a requirement for this course. All contant can be understood and studied without using the live code functionality. Students wishing to "skip" the codes can do so and focus on the theoretical content instead. 
* One way to set-up Jupyter is to install the Conda Python environment (https://www.anaconda.com/distribution/), which comes with Jupyter. See also instructions here: https://jupyter.org/
* For Octave (MATLAB syntax) notebooks users need to install Octave: https://www.gnu.org/software/octave/
* For Julia notebooks users could install Julia (https://julialang.org/) or use JuliaBox to run to notebooks directly in a browser (https://www.juliabox.com/). 
* The source for this notebook is found here: GitHub source: [https://github.com/Kevin-Mattheus-Moerman/NUIG_BME_402_6101](https://github.com/Kevin-Mattheus-Moerman/NUIG_BME_402_6101)


# Learning objectives
At the end of this lecture you should be able to:   

* Formulate Hooke's law in **matrix form**
* Derive spring equations using the **direct method**
* Derive spring equations using a **variational method**

# Hooke's law and the direct (stiffness) method
A two node spring system:   
![spring](spring_2Node.png)  
The force components at node 1 and 2 can be written:
$$f_{1x}=k(u_1-u_2)$$
$$f_{2x}=k(u_2-u_1)=-k(u_1-u_2)$$

It is conventient to use matrix notation allowing one to write the above as:
$$\begin{Bmatrix} f_{1x} \\ f_{2x} \end{Bmatrix}=\begin{bmatrix} k & -k \\ -k & k\end{bmatrix}\begin{Bmatrix} u_1 \\ u_2\end{Bmatrix}$$
Which in short form is written:
$$\begin{Bmatrix} F\end{Bmatrix}=\begin{bmatrix} K \end{bmatrix}\begin{Bmatrix} u\end{Bmatrix}$$
This introduces the notation $\begin{Bmatrix} \end{Bmatrix}$ and $\begin{bmatrix} \end{bmatrix}$, which represent a column array and a square array respectively.

## Methods for derivation of the finite element equations

* The key finite element equation:
$$\begin{Bmatrix} F\end{Bmatrix}=\begin{bmatrix} K \end{bmatrix}\begin{Bmatrix} u\end{Bmatrix}$$
* There are three main types of methods to do so:   
    * The **direct** (equilibrium) method   
        * Simple, intuitive
        * 1D problems
    * **Variational** methods
        * More general
        * Requires existance of a functional to minimize
    * **Weighted residual** methods (e.g. Galerkin)
        * Most general 
        * No functional for minization required
* The direct and variational method are presented in this course

## Matrix multiplication in finite element analysis (FEA)
* Multiplication of two rectangular matrices (summation implied over repeated indices):
$$c_{ij}=a_{ik}b_{kj}$$
* In FEA 
$$\begin{Bmatrix} F\end{Bmatrix}=\begin{bmatrix} K \end{bmatrix}\begin{Bmatrix} u\end{Bmatrix}$$
* Involves multiplication of a $n\times p$ matrix and a $n\times1$ array:
$$f_{i}=k_{ip}u_{p}$$
* Quasi-mnemonic to get $f_{i}$ "keep up" $\rightarrow k_{ip}u_{p}$

## Example: A three spring system and a known force \*
Consider the three spring system below. 
* The system consists of 3 spring elements and 4 nodes. 
* Node 1, 3, and 4 are constrained from moving. 
* Spring 2 and 3 act in parallel and both originate from node 2
* The spring stiffnesses are: $k_1=400$, $k_2=k_3=200$
* A force of 25 kN is applied to node 2 in the positive x-direction. 

**Assignment**: Use the direct stiffness method to derive the finite element equations

![3 spring example](springSet_3_force2.png)   
         
\*Based on example 2.3 of: Daryl Logan, _"A First Course in the Finite Element Methods"_ (page 51 in the 6th edition, page 52 in the 5th edition)

### Boundary and compatibility conditions
* The boundary conditions: 
$$u_1=u_3=u_4=0$$
* The compatibility condition:
$$u_2^{(1)}=u_2^{(2)}=u_2^{(3)}=u_2$$


### Setting up the element stiffness matrices
* Using $K^{(i)}=\begin{bmatrix} k_i & -k_i \\ -k_i & k_i\end{bmatrix} $, and $k_1=400$, $k_2=k_3=200$ we come to:

$K^{(1)}=\begin{bmatrix} 400 & -400 \\ -400 & 400\end{bmatrix}$, $K^{(2)}=\begin{bmatrix} 200 & -200 \\ -200 & 200\end{bmatrix}$, and $K^{(3)}=\begin{bmatrix} 200 & -200 \\ -200 & 200\end{bmatrix}$

In [1]:
kSet=[400 200 200]; # The spring stiffness value set
I=[1 -1; -1 1]; # Array for spawning stiffness matrices
k1=kSet[1]*I; # Element stiffness matrix 1
k2=kSet[2]*I; # Element stiffness matrix 2
k3=kSet[3]*I; # Element stiffness matrix 3
display(k1)
display(k2)
display(k3)

2×2 Array{Int64,2}:
  400  -400
 -400   400

2×2 Array{Int64,2}:
  200  -200
 -200   200

2×2 Array{Int64,2}:
  200  -200
 -200   200

### Superposition to assemble global stiffness matrix
Through superposition to global stiffness matrix can be assembled. The superposition is often written as:
$$K=\sum_{i=1}^{3} K^{(i)}$$
However, it should be noted this is not a summation. The below numerical implementation illustrates how **the indices of the nodes involved are used as indices into matrix $K$**, leading to:
$$\begin{bmatrix} K \end{bmatrix}=\begin{bmatrix} 400 & -400 & 0 & 0 \\ -400 & 800 & -200 & -200  \\ 0 & -200 & 200 & 0 \\ 0 & -200 & 0 & 200 \end{bmatrix}$$

In [2]:
K=zeros(4,4); #Initialize stiffness aray with all zeros
K[[1,2],[1,2]] .= K[[1,2],[1,2]] .+ k1; #Add element 1 contribution
K[[2,3],[2,3]] .= K[[2,3],[2,3]] .+ k2; #Add element 2 contribution
K[[2,4],[2,4]] .= K[[2,4],[2,4]] .+ k3; #Add element 3 contribution
display(K)

4×4 Array{Float64,2}:
  400.0  -400.0     0.0     0.0
 -400.0   800.0  -200.0  -200.0
    0.0  -200.0   200.0     0.0
    0.0  -200.0     0.0   200.0

### Solving the system
* Following derivation of the global stiffness matrix, and using $u_1=u_3=u_4=0$, the total system now becomes:
$$\begin{Bmatrix} F_1 \\ 25000 \\ F_3 \\ F_4 \end{Bmatrix}=\begin{bmatrix} 400 & -400 & 0 & 0 \\ -400 & 800 & -200 & -200  \\ 0 & -200 & 200 & 0 \\ 0 & -200 & 0 & 200 \end{bmatrix}\begin{Bmatrix} 0 \\ u_2 \\ 0 \\ 0\end{Bmatrix}$$
* Leading to: 


$$25000=\begin{bmatrix}-400 & 800 & -200 & -200 \end{bmatrix} \begin{Bmatrix} 0 \\ u_2 \\ 0 \\ 0\end{Bmatrix}\rightarrow 25000=800 u_2 \rightarrow u_2=\frac{25000}{800}$$

In [3]:
F2=25000; #Force at node 2
k22=K[2,2] #Stiffness matrix entry
u2=F2/k22 #Displacement array for node 2 and 3

31.25

### Compute force array F
Since all nodal diplacements are now known the full force array can now be computed from:
$$\begin{Bmatrix} F_1 \\ F_2 \\ F_3 \\ F_4 \end{Bmatrix}=\begin{bmatrix} 400 & -400 & 0 & 0 \\ -400 & 800 & -200 & -200  \\ 0 & -200 & 200 & 0 \\ 0 & -200 & 0 & 200 \end{bmatrix}\begin{Bmatrix} 0 \\ 31.25 \\ 0 \\ 0\end{Bmatrix}=\begin{Bmatrix} -12500 \\ 25000 \\ -6250 \\ -6250 \end{Bmatrix}$$

In [1]:
U=[0; u2; 0; 0] #Full displacement array
F=K*U #Compute force array

UndefVarError: UndefVarError: u2 not defined

### Computing element forces
The element force data can now be computed too from:
$$\begin{Bmatrix} F^{(i)} \end{Bmatrix}=\begin{bmatrix} k^{(i)}\end{bmatrix}\begin{Bmatrix} u^{(i)} \end{Bmatrix}$$   
  
E.g.:   

$$\begin{Bmatrix} f_{1x}^{(1)} \\ f_{2x}^{(1)} \end{Bmatrix}=\begin{bmatrix} k_1 & -k_1 \\ -k_1 & k_1\end{bmatrix}\begin{Bmatrix} u_1 \\ u_2\end{Bmatrix}$$
$$\begin{Bmatrix} f_{2x}^{(2)} \\ f_{3x}^{(2)} \end{Bmatrix}=\begin{bmatrix} k_2 & -k_2 \\ -k_2 & k_2\end{bmatrix}\begin{Bmatrix} u_2 \\ u_3\end{Bmatrix}$$
$$\begin{Bmatrix} f_{2x}^{(3)} \\ f_{4x}^{(3)} \end{Bmatrix}=\begin{bmatrix} k_3 & -k_3 \\ -k_3 & k_3\end{bmatrix}\begin{Bmatrix} u_2 \\ u_4\end{Bmatrix}$$

In [6]:
f1=k1*U[[1,2]]; #Element 1 forces
f2=k2*U[[2,3]]; #Element 2 forces
f3=k3*U[[2,4]]; #Element 3 forces
display(f1)
display(f2)
display(f3)

2-element Array{Float64,1}:
 -12500.0
  12500.0

2-element Array{Float64,1}:
  6250.0
 -6250.0

2-element Array{Float64,1}:
  6250.0
 -6250.0

# Mechanical applications: e.g. Hooke's law
$$F=ku$$   
  
$F$: Force in Newton (N)  
$k$: Spring stiffness constant (N/m)   
$u$: Spring extension (displacement of end point) (m)

![](HOOKE_LAW.png)

$$\begin{Bmatrix} f_{1} \\ f_{2} \end{Bmatrix}=\begin{bmatrix} k & -k \\ -k & k\end{bmatrix}\begin{Bmatrix} u_1 \\ u_2\end{Bmatrix}\rightarrow \begin{Bmatrix} F\end{Bmatrix}=\begin{bmatrix} K \end{bmatrix}\begin{Bmatrix} u\end{Bmatrix}$$


# Electrical applications: e.g. Ohm's law
$$V=RI$$   
  
$V$: Potential or voltage drop in volts (V)   
$R$: Resistance in Ohms ($\Omega$)   
$I$: Electric current in amperes (amps)   

![](OHM_LAW.png)

$$\begin{Bmatrix} V_{1} \\ V_{2} \end{Bmatrix}=\begin{bmatrix} r & -r \\ -r & r\end{bmatrix}\begin{Bmatrix} I_1 \\ I_2\end{Bmatrix}\rightarrow \begin{Bmatrix} V\end{Bmatrix}=\begin{bmatrix} R \end{bmatrix}\begin{Bmatrix} I\end{Bmatrix}$$

# Beyond 1D springs, introducing trusses

## The spring element
![spring](spring_2Node.png)     
* Behaviour Hooke's law for a spring: 
$$F=ku$$   
$$\begin{Bmatrix} f_{1} \\ f_{2} \end{Bmatrix}=k\begin{bmatrix} 1 & -1 \\ -1 & 1\end{bmatrix}\begin{Bmatrix} u_1 \\ u_2\end{Bmatrix}$$
* Spring is a 1D "line element"and has no thickness


## The truss element
![spring](truss1.png)  
* Behaviour Hooke's law for a bar: 
$$\sigma=E\epsilon=\frac{F}{A}$$   
$$\begin{Bmatrix} f_{1} \\ f_{2} \end{Bmatrix}=\frac{AE}{L}\begin{bmatrix} 1 & -1 \\ -1 & 1\end{bmatrix}\begin{Bmatrix} u_1 \\ u_2\end{Bmatrix}$$
* Truss (or bar) is a 1D element but has a **cross-sectional area** $A$

$\sigma$: Stress (Pa or $N/m^2$)   
$F$: Force (N)   
$E$: The material Young's Modulus (Pa or $N/m^2$)   
$\epsilon$: The strain in the material ([])   
$L$ truss lenght (m)

# Strain
* **strain** a relative metric for deformation: 
$$\epsilon=\frac{(u_2-u_1)}{L_0}$$    
* **stretch** or stretch-ratio, current ($L$) over initial length ($L_o$): 
$$\lambda=\frac{L}{L_0}=\frac{L_0+(u_2-u_1)}{L_0}=\frac{(u_2-u_1)}{L_0}+1$$      

* Different strain metrics exist, focus in this course is linear (or Biot) strain
    * Linear (or Biot) strain:
    $$\epsilon=\lambda-1=\frac{(u_2-u_1)}{L_0}$$
    * Green-Lagrange strain:
    $$\epsilon_{GL}=\frac{1}{2}(\lambda^2-1)$$
    * Logarithmic (or true or natural or Hencky) strain:
    $$\epsilon_{log}=\ln(\lambda)$$
    ![spring](strainsMoerman.jpg)  

# The truss (or bar) element 
![spring](truss2.png)  
* Stress (force acting over area) constitutive law: 
$$\sigma=E\epsilon=\frac{F}{A}$$      
* Force in the truss: $F=AE\epsilon$, therefore:
$$T=AE\frac{(u_2-u_1)}{L}=\frac{AE}{L}(u_2-u_1)$$
* The nodal forces:
$$f_{1}=-T=\frac{AE}{L}(u_1-u_2)$$
     
$$f_{2}=T=\frac{AE}{L}(-u_1+u_2)$$
* Expressed in matrix form:
$$\begin{Bmatrix} f_{1} \\ f_{2} \end{Bmatrix}=\frac{AE}{L}\begin{bmatrix} 1 & -1 \\ -1 & 1\end{bmatrix}\begin{Bmatrix} u_1 \\ u_2\end{Bmatrix}$$

## Example: A three spring system and a known force \*
Consider the three truss system below. 
* The system consists of 3 truss elements and 4 nodes. 
* Node 1, 3, and 4 are constrained from moving. 
* Truss 2 and 3 act in parallel and both originate from node 2
* All truss lengths $L_i=1$
* The truss Youngs Moduli are: $E_1=100$, $E_2=100$, and $E_3=200$
* The truss cross sectional areas are: $A_1=4$, $A_2=2$, and $A_3=1$
* A force of 25 kN is applied to node 2 in the positive x-direction. 

**Assignment**: Use the direct stiffness method to derive the finite element equations
![3 spring example](trussSet_3_force2.png)   

### Boundary and compatibility conditions
* The boundary conditions: 
$$u_1=u_3=u_4=0$$
* The compatibility condition:
$$u_2^{(1)}=u_2^{(2)}=u_2^{(3)}=u_2$$


### Setting up the element stiffness matrices
* Recognize: 
$$K^{(i)}=\frac{A^{(i)}E^{(i)}}{L^{(i)}}\begin{bmatrix} 1 & -1 \\ -1 & 1\end{bmatrix}=k^{(i)}\begin{bmatrix} 1 & -1 \\ -1 & 1\end{bmatrix}$$   
   
$$k^{(1)}=\frac{A^{(1)}E^{(1)}}{L^{(1)}}=\frac{4*100}{1}=400$$
$$k^{(2)}=\frac{A^{(2)}E^{(2)}}{L^{(2)}}=\frac{2*100}{1}=200$$
$$k^{(3)}=\frac{A^{(3)}E^{(3)}}{L^{(3)}}=\frac{1*200}{1}=200$$

* Using $K^{(i)}=\begin{bmatrix} k_i & -k_i \\ -k_i & k_i\end{bmatrix} $, and $k_1=400$, $k_2=k_3=200$ we come to:

$K^{(1)}=\begin{bmatrix} 400 & -400 \\ -400 & 400\end{bmatrix}$, $K^{(2)}=\begin{bmatrix} 200 & -200 \\ -200 & 200\end{bmatrix}$, and $K^{(3)}=\begin{bmatrix} 200 & -200 \\ -200 & 200\end{bmatrix}$

### Superposition to assemble global stiffness matrix
Through superposition to global stiffness matrix can be assembled. The superposition is often written as:
$$K=\sum_{i=1}^{3} K^{(i)}$$
However, it should be noted this is not a summation. The below numerical implementation illustrates how **the indices of the nodes involved are used as indices into matrix $K$**, leading to:
$$\begin{bmatrix} K \end{bmatrix}=\begin{bmatrix} 400 & -400 & 0 & 0 \\ -400 & 800 & -200 & -200  \\ 0 & -200 & 200 & 0 \\ 0 & -200 & 0 & 200 \end{bmatrix}$$

# Summary 
* A truss or bar element is a line-like element with an associated area and initial length
![spring](truss1.png)  
* Mechanical behaviour here linear elastic as per Hooke's law: 
$$\sigma=E\epsilon=\frac{F}{A}$$   
* Element equations and assembly using direct method comparible to spring equations with $k^{(i)}=\frac{A^{(i)}E^{(i)}}{L^{(i)}}$
* Hence *effective stiffness* of a structure is due to material constants ($E$) **AND** geometry ($A$, and $L$)
* For instance: 
$$\begin{Bmatrix} f_{1} \\ f_{2} \end{Bmatrix}=\frac{AE}{L}\begin{bmatrix} 1 & -1 \\ -1 & 1\end{bmatrix}\begin{Bmatrix} u_1 \\ u_2\end{Bmatrix}$$